# SETUP PRE-REQUISITES

In [ ]:
import os
import platform
import sys

import tensorflow as tf
from tensorflow.keras import layers

import keras
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.layers import Dense, Flatten, Input
from keras.models import Model

def escape():
    sys.exit()

In [ ]:
%matplotlib widget

In [ ]:
print(f"Python Platform: {platform.platform()}")
print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tf.keras.__version__}")
print()
print(f"Python {sys.version}")

# SETUP GPU 

1. The TF_GPU_ALLOCATOR environment variable is used to specify the GPU memory allocator that TensorFlow should use. The cuda_malloc_async allocator is an asynchronous memory allocator that is optimized for use with NVIDIA GPUs.

In [ ]:
os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"

2. The `os.environ['TF_FORCE_GPU_ALLOW_GROWTH']` assignment sets an environment variable called `TF_FORCE_GPU_ALLOW_GROWTH` in the operating system. This environment variable tells TensorFlow to dynamically grow the GPU memory usage as needed, rather than allocating all of the GPU memory at the start of the TensorFlow session.

In [ ]:
#os.environ['TF_FORCE_GPU_ALLOW_GROWTH']= 'true'

## Setup INFO level

In [ ]:
tf.get_logger().setLevel("INFO")
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}

## Setup Dynamic Memory Allocation

### Method 1: This enables the dynamic allocation of GPU memory. 

In [ ]:
#gpus = tf.config.experimental.list_physical_devices("GPU")
#if gpus:
#    try:
#        for gpu in gpus:
#            tf.config.experimental.set_memory_growth(gpu, True)
#    except RuntimeError as e:
#        print(e)

### Method 2: This enables the dynamic allocation of GPU memory. 

In [ ]:
# Ref: https://www.tensorflow.org/guide/gpu#limiting_gpu_memory_growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
# Restrict TensorFlow to only allocate 1GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)]) # Notice here
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
# Virtual devices must be set before GPUs have been initialized
    print(e)

## Checks for the GPU available and lists it

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices("GPU")))

In [ ]:
from tensorflow.python.client import device_lib

print("Num GPUs Available: ", len(tf.config.list_physical_devices("GPU")))
device_lib.list_local_devices()

# MAIN CODE

In [ ]:
# import os
# import sys

# import keras
# import matplotlib as mpl
# import matplotlib.pyplot as plt
# import numpy as np
# from keras.applications.vgg16 import VGG16
# from keras.applications.vgg19 import VGG19
# from keras.layers import Dense, Flatten, Input
# from keras.models import Model

# def escape():
#     sys.exit()

In [ ]:
import_directory = "/spectre-code/dataset/PCAP-NPY/"
counter = 0
data_array = np.empty((0, 2))
files = os.listdir(import_directory)
for file in files:
    print("Opening File : ", file)
    data_set = np.load(import_directory + file, allow_pickle=True)
    data_array = np.vstack((data_array, data_set))

# separated = np.array([x for x in data_array if x[1] == 0])

In [ ]:
# np.random.shuffle(separated)
np.random.shuffle(data_array)
print("ok")

In [ ]:
img_row = 50
img_col = 50

N = np.shape(data_array)[0]
# train_test_split_percentage = 0.75
train_test_split_percentage = 0.60

X_train = data_array[: int(N * train_test_split_percentage), 0]
X_test = data_array[int(N * train_test_split_percentage) :, 0]

X_train = np.array([x.reshape(img_row, img_col, 3) for x in X_train])
X_test = np.array([x.reshape(img_row, img_col, 3) for x in X_test])

In [ ]:
y_train = data_array[: int(N * train_test_split_percentage), 1]
y_test = data_array[int(N * train_test_split_percentage) :, 1]

y_train = np.array([[x] for x in y_train])
y_test = np.array([[x] for x in y_test])

In [ ]:
model_vgg19_conv = VGG19(
    include_top=False, weights="imagenet", input_shape=(img_col, img_row, 3)
)

In [ ]:
# ploting images for data
%matplotlib inline
n = 40  # how many digits we will display
plt.figure(figsize=(400, 150))
for i in range(6, 11):
    # display original
    ax = plt.subplot(1, n, i + 1)
    plt.imshow(X_train[i])
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()
plt.close()

In [ ]:
for layer in model_vgg19_conv.layers:
    layer.trainable = False
    print(layer.name)

In [ ]:
# x = model_vgg19_conv.output
# x = Flatten()(x)
# x = Dense(128, activation='relu')(x)
# x = Dense(1, activation='sigmoid', name='predictions')(x)
# my_model = Model(inputs=model_vgg19_conv.input, outputs=x)
# my_model.summary()

# Revision 1
# x = model_vgg19_conv.output
# x = Flatten()(x)
# x = Dense(128, activation='relu')(x)
# x = Dense(80, activation='relu')(x)
# x = Dense(1, activation='sigmoid', name='predictions')(x)
# my_model = Model(inputs=model_vgg19_conv.input, outputs=x)
# my_model.summary()

# Revision 2
x = model_vgg19_conv.output
x = layers.Flatten()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dense(1, activation="sigmoid", name="predictions")(x)
my_model = tf.keras.Model(inputs=model_vgg19_conv.input, outputs=x)
my_model.summary()

In [ ]:
my_model.compile(
    loss=keras.losses.binary_crossentropy,
    optimizer=keras.optimizers.RMSprop(),
    metrics=["accuracy"],
)

batch_size = 5
epochs = 3

In [ ]:
hist = my_model.fit(
    X_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    verbose=1,
    #validation_split=0.2,
    validation_split=0.4,
)

In [ ]:
score = my_model.evaluate(X_test, y_test, verbose=0)

In [ ]:
print("Test loss:", score[0])
print("Test accuracy:", score[1])

In [ ]:
training_loss = hist.history["loss"]
val_loss = hist.history["val_loss"]
training_acc = hist.history["acc"]
val_acc = hist.history["val_acc"]
xc = range(epochs)

plt.figure(1, figsize=(7, 5))
plt.plot(xc, training_loss)
plt.plot(xc, val_loss)
plt.xlabel("No. of Epochs")
plt.ylabel("loss")
plt.title("Training Loss vs Validation Loss")
plt.grid(True)
plt.legend(["Train", "Val"])

plt.figure(2, figsize=(7, 5))
plt.plot(xc, training_acc)
plt.plot(xc, val_acc)
plt.xlabel("No. of Epochs")
plt.ylabel("Accuracy")
plt.title("Training Accuracy vs Validation Accuracy")
plt.grid(True)
plt.legend(["Train", "Val"], loc=4)

In [ ]:
checking_occurences = [x for x in y_test if x == 1]
print("total length = ", len(y_test))
print("Anomolies = ", len(checking_occurences))
print("Normal = ", (len(y_test) - len(checking_occurences)))

n = len(y_test) - len(checking_occurences)
a = len(checking_occurences)

label = ["Normal", "Anomaly"]
samples = [n, a]

default_dpi = mpl.rcParamsDefault["figure.dpi"]
mpl.rcParams["figure.dpi"] = default_dpi * 1.5

# this is for plotting purpose
index = np.arange(len(label))
plt.bar(index, samples)
plt.xlabel("Payload Types", fontsize=10)
plt.ylabel("Samples", fontsize=10)
plt.xticks(index, label, fontsize=10, rotation=0)
plt.title("Samples identified by VGG19")
plt.show()

In [ ]:
predictions = my_model.predict(X_test)
print("Predictions = ", predictions)

In [ ]:
rounded = [[round(x[0])] for x in predictions]
from sklearn.metrics import f1_score

result = f1_score(y_test, rounded, average=None, labels=[0, 1])
print("F1 = ", result)

In [ ]:
result = f1_score(y_test, rounded, average="macro")
print("F1 macro = ", result)

In [ ]:
result = f1_score(y_test, rounded, average="micro")
print("F1 Micro= ", result)

In [ ]:
result = f1_score(y_test, rounded, average="weighted")
print("F1 Weighted= ", result)

In [ ]:
import sklearn.metrics

target_names = ["normal", "anomaly"]
print(
    sklearn.metrics.classification_report(
        y_test, rounded, labels=[0, 1], target_names=target_names
    )
)

In [ ]:
my_model.save("FYP_Finalhd5.hd5")
my_model.save("FYP_Finalh5.h5")

In [ ]:
from read_activations import display_activations, get_activations

v_index = 4
x_valid = X_train[:5]
a = get_activations(my_model, x_valid[v_index : v_index + 1], print_shape_only=True)
display_activations(a)